# Fondamenti di elaborazione immagini

## Effettuiamo l'import delle librerie utilizzate nell'esercitazione.

In [ ]:
import cv2
import numpy as np
import matplotlib as mapli
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

Di seguito i riferimenti alle pagine di documentazione, sempre utiliti:

* Rif: [numpy](https://numpy.org/doc/stable/)
* Rif: [opencv](https://docs.opencv.org/)
* Rif: [matplotlib](https://matplotlib.org/stable/index.html)

Aggiungiamo alcune funzioni di utilita' per semplificare la scrittura del codice.

In [ ]:
def rgb(image : np.array) -> np.array:
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def gray(image : np.array) -> np.array:
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def grid(images : list[np.array], rows : int, cols : int, size : int, colors : list[str] = None) -> None:
    fig = plt.figure(figsize=(size,size))
    grid = ImageGrid(fig, 111, nrows_ncols=(rows, cols), axes_pad=0.1)

    if colors is not None:
        counter = 0
        for ax, im in zip(grid, images):
            ax.imshow(im, cmap=colors[counter])
            counter = (counter + 1) % len(colors)
        plt.show()
    else:
        for ax, im in zip(grid, images):
            ax.imshow(im)
        plt.show()

## _L'istogramma e' un potente strumento messo a disposizione per l'analisi delle immagini._

Quest'ultimo, permette di mostrare quanti pixel (asse y) possiedono uno specifico valore (asse x) in un semplice grafico a linee o barre. Di seguito carichiamo un'immagine colore e ne mostriamo l'istogramma usando due metodi che, come visto in precedenza, sfruttano _opencv_ e _numpy_.

In [ ]:
bgr_image = cv2.imread('./imgs/kitten.png', cv2.IMREAD_COLOR)  # carico l'immagine colore.
color = ('b','g','r')

In [ ]:
b_histogram = cv2.calcHist([bgr_image], [0], None, [256],[0,256])   # calcolo l'istogramma per il piano blu.
g_histogram = cv2.calcHist([bgr_image], [1], None, [256],[0,256])   # calcolo l'istogramma per il piano verde.
r_histogram = cv2.calcHist([bgr_image], [2], None, [256],[0,256])   # calcolo l'istogramma per il piano rosso.
hists = [b_histogram, g_histogram, r_histogram]

plt.figure(figsize=(20,8))                                          # disegno l'istogramma.
for i in range(len(color)):
    plt.plot(hists[i], color=color[i])
    plt.xlim([0, 256])
    plt.ylim([0, 22000])
plt.show()

Tramite la funzione _calcHist_ di _opencv_ e' possibile calcolare l'istogramma di un piano immagine e con la funzione _plot_ di matplotlib e' possibile mostrarlo. La parametrizzazione di _calcHist_ permette di utilizzare piu' di un'immagine e di un piano ma per l'utilizzo che a noi interessa, questo e' sufficiente. Vedere i riferimenti.

* Rif: [calcHist](https://docs.opencv.org/4.7.0/d6/dc7/group__imgproc__hist.html#ga4b2b5fd75503ff9e6844cc4dcdaed35d)
* Rif: [plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html)

Allo stesso modo, possiamo sfruttare la libreria _numpy_ lavorando direttamente sulle matrici.

In [ ]:
b_histogram, _ = np.histogram(bgr_image[:,:,0].ravel(), 256, [0,256])  # calcolo l'istogramma per il piano blu.
g_histogram, _ = np.histogram(bgr_image[:,:,1].ravel(), 256, [0,256])  # calcolo l'istogramma per il piano verde.
r_histogram, _ = np.histogram(bgr_image[:,:,2].ravel(), 256, [0,256])  # calcolo l'istogramma per il piano rosso.
hists = [b_histogram, g_histogram, r_histogram]

plt.figure(figsize=(20,8))                                              # disegno l'istogramma.
for i in range(len(color)):
    plt.plot(hists[i], color=color[i])
    plt.xlim([0, 256])
    plt.ylim([0, 22000])
plt.show()

In questo caso abbiamo utilizzato _numpy_ sfruttando i metodi _ravel_ e _histogram_. Con il primo non si fa altro che prendere un'immagine (nel nostro caso un piano) e lo si trasforma in un vettore di valori. Con il secondo metodo, si analizza il vettore di valori al fine di trovare l'istogramma.

* Rif: [ravel](https://numpy.org/doc/stable/reference/generated/numpy.ravel.html)
* Rif: [histogram](https://numpy.org/doc/stable/reference/generated/numpy.histogram.html)

## _L'analisi di un istogramma permette spesso di trarre rapide conclusioni quali, ad esempio, le differenze di luminosita' fra immagini._

Carichiamo ad esempio due immagini con una differenza palese di luminosita' e cerchiamo di ritrovare la stessa informazione negli istogrammi.

In [ ]:
light_img = cv2.imread('./imgs/light.png', cv2.IMREAD_COLOR)   # carico un'immagine chiara.
dark_img = cv2.imread('./imgs/dark.png', cv2.IMREAD_COLOR)     # carico un'immagine scura.
grid([rgb(light_img),rgb(dark_img)], 1, 2, 20)

In [ ]:
light_b_histogram, _ = np.histogram(light_img[:,:,0].ravel(), 256, [0,256]) # istogramma del blu.
light_g_histogram, _ = np.histogram(light_img[:,:,1].ravel(), 256, [0,256]) # istogramma del verde.
light_r_histogram, _ = np.histogram(light_img[:,:,2].ravel(), 256, [0,256]) # istogramma del rosso.
light_hists = [light_b_histogram, light_g_histogram, light_r_histogram]

dark_b_histogram, _ = np.histogram(dark_img[:,:,0].ravel(), 256, [0,256])   # istogramma del blu.
dark_g_histogram, _ = np.histogram(dark_img[:,:,1].ravel(), 256, [0,256])   # istogramma del verde.
dark_r_histogram, _ = np.histogram(dark_img[:,:,2].ravel(), 256, [0,256])   # istogramma del rosso.
dark_hists = [dark_b_histogram, dark_g_histogram, dark_r_histogram]

plt.subplots(1, 2, figsize=(20, 8))
plt.subplot(1, 2, 1)
for i in range(len(color)):
    plt.plot(light_hists[i], color=color[i])
plt.title('light')
plt.subplot(1, 2, 2)
for i in range(len(color)):
    plt.plot(dark_hists[i], color=color[i])
plt.title('dark')
plt.show()

Con i metodi _subplots_ e _subplot_ di _matplotlib_ e' possibile configurare la figura da disegnare permettendo di mostrare piu' grafici in una modalita' a griglia. In questo caso, il confronto fra gli istogrammi mostra il palese sbilanciamento verso i colori chiari (vicini a 255) nell'immagine chiara e verso i colori scuri (vicino a 0) nell'immagine scura.

* Rif: [subplots](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html)
* Rif: [subplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplot.html)

## _Altrettando possibile e' vedere se immagini contengono gruppi di pixel fra loro facilmente separabili._

Proviamo, ad esempio, ad analizzare l'istogramma di una immagine in cui due oggetti palesemente distinti, sono distinguibili anche leggendo i valori dell'istogramma.

In [ ]:
leveled_img = cv2.imread('./imgs/thresholds/gray.png', cv2.IMREAD_GRAYSCALE)
plt.imshow(leveled_img)
plt.show()

In [ ]:
hist, _ = np.histogram(leveled_img.ravel(), 256, [0,256])   # calcolo l'istogramma.
plt.figure(figsize=(20,8))
plt.plot(hist)
plt.show()

Dall'istogramma vediamo, infatti, come esistano due gruppi di pixel con uno specifico valore che, in questo caso, rappresentano due oggetti distinti: il triangolo e il quadrato.